## Подготовка модели для web-приложения

In [1]:

import numpy as np
import pandas as pd
import pickle

from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor


In [2]:
# Загружаем очищенные от выбросов данные
df = pd.read_csv('../../data/clean.csv')
print(df.shape)
df.head()

(936, 13)


,"Количество отвердителя, м.%","Содержание эпоксидных групп, %","Температура вспышки, С","Потребление смолы, г/м2","Угол нашивки, град",Шаг нашивки,Плотность нашивки,"Плотность, кг/м3","Поверхностная плотность, г/м2","Модуль упругости, ГПа",Соотношение матрица-наполнитель,"Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа"
0,50.00,23.750000,284.615385,220.0,0,4.0,60.0,2030.0,210.0,738.736842,1.857143,70.0,3000.0
1,129.00,21.250000,300.000000,220.0,0,5.0,47.0,2030.0,210.0,738.736842,1.857143,70.0,3000.0
2,111.86,22.267857,284.615385,220.0,0,5.0,57.0,2030.0,210.0,753.000000,2.771331,70.0,3000.0
3,111.86,22.267857,284.615385,220.0,0,5.0,60.0,2000.0,210.0,748.000000,2.767918,70.0,3000.0
4,111.86,22.267857,284.615385,220.0,0,5.0,70.0,1910.0,210.0,807.000000,2.569620,70.0,3000.0


In [66]:
# Разделяем X и y

y = df['Модуль упругости при растяжении, ГПа']

X = df.drop(columns=['Модуль упругости при растяжении, ГПа',
                     'Прочность при растяжении, МПа'])
X.shape

Index(['Количество отвердителя, м.%', 'Содержание эпоксидных групп, %',
       'Температура вспышки, С', 'Потребление смолы, г/м2',
       'Угол нашивки, град', 'Шаг нашивки', 'Плотность нашивки',
       'Плотность, кг/м3', 'Поверхностная плотность, г/м2',
       'Модуль упругости, ГПа', 'Соотношение матрица-наполнитель'],
      dtype='object')

In [32]:
# Переводим в массив, в нужную размерность

X = np.array(X)
y = np.array(y)

y = y.reshape(-1, 1)

In [33]:
# Масштабируем

scaler_X = MinMaxScaler()
X_scl = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scl = scaler_y.fit_transform(y)

In [34]:
# Загружаем гиперпараметры для модели

best_param_xgb = pickle.load(open('../../notebooks/evaluation/best_param_xgb.pickle', 'rb'))
best_param_xgb

{'booster': 'dart',
 'colsample_bytree': 0.5,
 'learning_rate': 0.02,
 'max_depth': 10,
 'n_estimators': 10,
 'n_jobs': -1,
 'subsample': 0.7}

In [35]:
# Создаем и обучаем модель

best_xgb = XGBRegressor(random_state=0, **best_param_xgb)
best_xgb.fit(X_scl, y_scl)

XGBRegressor(base_score=None, booster='dart', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.02, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=-1, num_parallel_tree=None, predictor=None,
             random_state=0, ...)

In [36]:
# Подготовка входного значения для предсказания

X_test = X[0]
X_test

array([5.00000000e+01, 2.37500000e+01, 2.84615385e+02, 2.20000000e+02,
       0.00000000e+00, 4.00000000e+00, 6.00000000e+01, 2.03000000e+03,
       2.10000000e+02, 7.38736842e+02, 1.85714286e+00])

In [37]:
X_test.reshape(1, -1)

array([[5.00000000e+01, 2.37500000e+01, 2.84615385e+02, 2.20000000e+02,
        0.00000000e+00, 4.00000000e+00, 6.00000000e+01, 2.03000000e+03,
        2.10000000e+02, 7.38736842e+02, 1.85714286e+00]])

In [38]:
# Масштабирование входного значения

X_test_scl = scaler_X.transform(X_test.reshape(1, -1))
X_test_scl

array([[0.07915272, 0.60743528, 0.50916426, 0.52922146, 0.        ,
        0.28933398, 0.55715613, 0.65109743, 0.16223008, 0.44706097,
        0.27476816]])

In [39]:
# Предсказание

y_pred_scl = best_xgb.predict(X_test_scl)
y_pred_scl

array([0.4867716], dtype=float32)

In [40]:
# Обратное масштабирование прогноза

y_pred = scaler_y.inverse_transform(y_pred_scl.reshape(-1, 1))
y_pred

array([[73.275375]], dtype=float32)

Сохранение модели и масштабаторов

In [43]:
# Обученная модель

pickle.dump(best_xgb, open('../model/model_xgb.pkl', 'wb'))

# Масштабатор X
pickle.dump(scaler_X, open('../model/scaler_X.pkl', 'wb'))

# Масштабатор y
pickle.dump(scaler_y, open('../model/scaler_y.pkl', 'wb'))

Функция для обработки данных и вычислений в приложении

In [63]:

def get_prediction(values):
    '''Функция для обработки данных и вычислений'''
    
    # загрузка модели
    model = pickle.load(open('../model/model_xgb.pkl', 'rb'))
    
    # загрузка масштабаторов
    scaler_X = pickle.load(open('../model/scaler_X.pkl', 'rb'))
    scaler_y = pickle.load(open('../model/scaler_y.pkl', 'rb'))
    
    # масштабирование входных значений
    X_scl = scaler_X.transform(values.reshape(1, -1))
    
    # прогноз
    y_pred_scl = model.predict(X_scl)
    
    # обратное масштабирование прогноза
    y_pred = scaler_y.inverse_transform(y_pred_scl.reshape(-1, 1))
    y_pred = round(float(y_pred), 2)
    
    return y_pred
    

Проверка работы функции

In [64]:
# входное значение для предсказания
values = X[2]
values

array([ 111.86      ,   22.26785714,  284.61538462,  220.        ,
          0.        ,    5.        ,   57.        , 2030.        ,
        210.        ,  753.        ,    2.77133106])

In [65]:
prediction = get_prediction(values)
prediction

73.19